In [ ]:
import torch

from data import data_helper
from data.data_helper import available_datasets
from models import model_factory
from optimizer.optimizer_helper import get_optim_and_scheduler
from torch.nn import functional as F
from torch import nn

from utils.Logger import Logger

import torchvision
import matplotlib.pyplot as plt

from train_jigsaw import Trainer
from utils import vis

class Container():
    pass

args = Container()
args.batch_size = 128
args.n_classes = 7
args.learning_rate = 0.001
args.epochs = 30
args.network = "caffenet"
args.val_size = 0.1
args.tf_logger = True
args.folder_name = "grid_search"
args.jigsaw_n_classes = 31
args.classify_only_sane = True
args.train_all = True
# args.jig_weight = 0.9
# args.bias_whole_image = 0.8
args.TTA = False
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jig_weight in [0.9, 1.1, 1.3]: 
    for args.bias_whole_image in [0.4, 0.6, 0.8]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()


cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 7150, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/grid_search/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.9_TAll_bias0.4_classifyOnlySane_90
New epoch - lr: 0.001
0/55 of epoch 1/30 jigsaw : 3.505261, class : 1.979711 - acc jigsaw : 0.039062, class : 0.164062 [bs:128]
30/55 of epoch 1/30 jigsaw : 2.090324, class : 0.558919 - acc jigsaw : 0.460938, class : 0.664062 [bs:128]
Accuracies on val: jigsaw : 0.997478, class : 0.819672
Accuracies on test: jigsaw : 0.993164, class : 0.626465
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 2.053485, class : 0.377757 - acc jigsaw : 0.429688, class : 0.648438 [bs:128]
30/55 of epoch 2/30 jigsaw : 1.882534, class : 0.451338 - acc jigsaw : 0.453125, class : 0.648438 [bs:128]
Accuracies on val: jigsaw : 0.997478, class : 0.853720
Accuracies on test: j

Accuracies on val: jigsaw : 1.000000, class : 0.923077
Accuracies on test: jigsaw : 0.991699, class : 0.664551
New epoch - lr: 0.001
0/55 of epoch 24/30 jigsaw : 0.208083, class : 0.017021 - acc jigsaw : 0.953125, class : 0.601562 [bs:128]
30/55 of epoch 24/30 jigsaw : 0.175242, class : 0.021089 - acc jigsaw : 0.937500, class : 0.671875 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.919294
Accuracies on test: jigsaw : 0.993652, class : 0.625977
New epoch - lr: 0.0001
0/55 of epoch 25/30 jigsaw : 0.147734, class : 0.039880 - acc jigsaw : 0.960938, class : 0.687500 [bs:128]
30/55 of epoch 25/30 jigsaw : 0.247872, class : 0.033228 - acc jigsaw : 0.921875, class : 0.648438 [bs:128]
Accuracies on val: jigsaw : 0.998739, class : 0.939470
Accuracies on test: jigsaw : 0.990723, class : 0.642578
New epoch - lr: 0.0001
0/55 of epoch 26/30 jigsaw : 0.178960, class : 0.056726 - acc jigsaw : 0.937500, class : 0.671875 [bs:128]
30/55 of epoch 26/30 jigsaw : 0.404301, class : 0.032955 - acc

30/55 of epoch 16/30 jigsaw : 0.250107, class : 0.127432 - acc jigsaw : 0.906250, class : 0.578125 [bs:128]
Accuracies on val: jigsaw : 0.997478, class : 0.938209
Accuracies on test: jigsaw : 0.986328, class : 0.655762
New epoch - lr: 0.001
0/55 of epoch 17/30 jigsaw : 0.286793, class : 0.165351 - acc jigsaw : 0.906250, class : 0.687500 [bs:128]
30/55 of epoch 17/30 jigsaw : 0.231887, class : 0.245642 - acc jigsaw : 0.921875, class : 0.625000 [bs:128]
Accuracies on val: jigsaw : 0.998739, class : 0.943253
Accuracies on test: jigsaw : 0.994141, class : 0.672363
New epoch - lr: 0.001
0/55 of epoch 18/30 jigsaw : 0.230260, class : 0.069030 - acc jigsaw : 0.914062, class : 0.632812 [bs:128]
30/55 of epoch 18/30 jigsaw : 0.261643, class : 0.061062 - acc jigsaw : 0.929688, class : 0.687500 [bs:128]
Accuracies on val: jigsaw : 0.998739, class : 0.928121
Accuracies on test: jigsaw : 0.994141, class : 0.623535
New epoch - lr: 0.001
0/55 of epoch 19/30 jigsaw : 0.217045, class : 0.097465 - acc j

Accuracies on test: jigsaw : 0.991699, class : 0.630371
New epoch - lr: 0.001
0/55 of epoch 9/30 jigsaw : 0.499816, class : 0.312848 - acc jigsaw : 0.867188, class : 0.625000 [bs:128]
30/55 of epoch 9/30 jigsaw : 0.402088, class : 0.156763 - acc jigsaw : 0.890625, class : 0.601562 [bs:128]
Accuracies on val: jigsaw : 0.998739, class : 0.890290
Accuracies on test: jigsaw : 0.994629, class : 0.594727
New epoch - lr: 0.001
0/55 of epoch 10/30 jigsaw : 0.472488, class : 0.131811 - acc jigsaw : 0.867188, class : 0.625000 [bs:128]
30/55 of epoch 10/30 jigsaw : 0.485923, class : 0.178765 - acc jigsaw : 0.851562, class : 0.640625 [bs:128]
Accuracies on val: jigsaw : 1.000000, class : 0.911728
Accuracies on test: jigsaw : 0.990234, class : 0.639160
New epoch - lr: 0.001
0/55 of epoch 11/30 jigsaw : 0.404785, class : 0.155365 - acc jigsaw : 0.867188, class : 0.625000 [bs:128]
30/55 of epoch 11/30 jigsaw : 0.343547, class : 0.123263 - acc jigsaw : 0.914062, class : 0.609375 [bs:128]
Accuracies on

30/53 of epoch 1/30 jigsaw : 2.063115, class : 0.678075 - acc jigsaw : 0.476562, class : 0.562500 [bs:128]
Accuracies on val: jigsaw : 0.997379, class : 0.841415
Accuracies on test: jigsaw : 0.992747, class : 0.577218
New epoch - lr: 0.001
0/53 of epoch 2/30 jigsaw : 1.998079, class : 0.516399 - acc jigsaw : 0.484375, class : 0.625000 [bs:128]
30/53 of epoch 2/30 jigsaw : 1.765915, class : 0.457581 - acc jigsaw : 0.515625, class : 0.554688 [bs:128]
Accuracies on val: jigsaw : 0.998689, class : 0.849279
Accuracies on test: jigsaw : 0.990614, class : 0.667662
New epoch - lr: 0.001
0/53 of epoch 3/30 jigsaw : 1.490450, class : 0.328405 - acc jigsaw : 0.601562, class : 0.679688 [bs:128]
30/53 of epoch 3/30 jigsaw : 1.393684, class : 0.627074 - acc jigsaw : 0.593750, class : 0.664062 [bs:128]
Accuracies on val: jigsaw : 0.988204, class : 0.858453
Accuracies on test: jigsaw : 0.970563, class : 0.687713
New epoch - lr: 0.001
0/53 of epoch 4/30 jigsaw : 1.305823, class : 0.761344 - acc jigsaw 

...

30/53 of epoch 10/30 jigsaw : 0.440516, class : 0.176225 - acc jigsaw : 0.890625, class : 0.578125 [bs:128]
Accuracies on val: jigsaw : 0.997379, class : 0.893840
Accuracies on test: jigsaw : 0.978669, class : 0.705631
New epoch - lr: 0.001
0/53 of epoch 11/30 jigsaw : 0.439442, class : 0.219179 - acc jigsaw : 0.867188, class : 0.609375 [bs:128]
30/53 of epoch 11/30 jigsaw : 0.364096, class : 0.198548 - acc jigsaw : 0.898438, class : 0.648438 [bs:128]
Accuracies on val: jigsaw : 0.994758, class : 0.913499
Accuracies on test: jigsaw : 0.978242, class : 0.671075
New epoch - lr: 0.001
0/53 of epoch 12/30 jigsaw : 0.376582, class : 0.115507 - acc jigsaw : 0.906250, class : 0.679688 [bs:128]
30/53 of epoch 12/30 jigsaw : 0.492634, class : 0.040276 - acc jigsaw : 0.859375, class : 0.648438 [bs:128]
Accuracies on val: jigsaw : 0.998689, class : 0.914810
Accuracies on test: jigsaw : 0.981229, class : 0.692833
New epoch - lr: 0.001
0/53 of epoch 13/30 jigsaw : 0.404609, class : 0.115262 - acc j

Accuracies on test: jigsaw : 0.992814, class : 0.857485
New epoch - lr: 0.001
0/58 of epoch 3/30 jigsaw : 1.900042, class : 0.592995 - acc jigsaw : 0.500000, class : 0.585938 [bs:128]
30/58 of epoch 3/30 jigsaw : 1.356954, class : 0.404697 - acc jigsaw : 0.656250, class : 0.609375 [bs:128]
Accuracies on val: jigsaw : 0.993976, class : 0.848193
Accuracies on test: jigsaw : 0.983832, class : 0.866467
New epoch - lr: 0.001
0/58 of epoch 4/30 jigsaw : 1.258359, class : 0.583836 - acc jigsaw : 0.617188, class : 0.664062 [bs:128]
30/58 of epoch 4/30 jigsaw : 1.238866, class : 0.490515 - acc jigsaw : 0.640625, class : 0.585938 [bs:128]
Accuracies on val: jigsaw : 0.995181, class : 0.863855
Accuracies on test: jigsaw : 0.986826, class : 0.839521
New epoch - lr: 0.001
0/58 of epoch 5/30 jigsaw : 1.181474, class : 0.438769 - acc jigsaw : 0.648438, class : 0.562500 [bs:128]
30/58 of epoch 5/30 jigsaw : 0.749509, class : 0.323780 - acc jigsaw : 0.750000, class : 0.726562 [bs:128]
Accuracies on val

Accuracies on val: jigsaw : 0.997590, class : 0.910843
Accuracies on test: jigsaw : 0.996407, class : 0.874850
New epoch - lr: 0.0001
0/58 of epoch 27/30 jigsaw : 0.259304, class : 0.084737 - acc jigsaw : 0.914062, class : 0.671875 [bs:128]
30/58 of epoch 27/30 jigsaw : 0.160170, class : 0.022287 - acc jigsaw : 0.976562, class : 0.695312 [bs:128]
Accuracies on val: jigsaw : 0.997590, class : 0.909639
Accuracies on test: jigsaw : 0.996407, class : 0.883234
New epoch - lr: 0.0001
0/58 of epoch 28/30 jigsaw : 0.254263, class : 0.022730 - acc jigsaw : 0.898438, class : 0.664062 [bs:128]
30/58 of epoch 28/30 jigsaw : 0.149756, class : 0.042602 - acc jigsaw : 0.945312, class : 0.695312 [bs:128]
Accuracies on val: jigsaw : 0.997590, class : 0.910843
Accuracies on test: jigsaw : 0.996407, class : 0.887425
New epoch - lr: 0.0001
0/58 of epoch 29/30 jigsaw : 0.258264, class : 0.004246 - acc jigsaw : 0.929688, class : 0.617188 [bs:128]
30/58 of epoch 29/30 jigsaw : 0.107958, class : 0.056528 - ac

30/58 of epoch 19/30 jigsaw : 0.314907, class : 0.057083 - acc jigsaw : 0.921875, class : 0.562500 [bs:128]
Accuracies on val: jigsaw : 0.998795, class : 0.893976
Accuracies on test: jigsaw : 0.992814, class : 0.871856
New epoch - lr: 0.001
0/58 of epoch 20/30 jigsaw : 0.182427, class : 0.038048 - acc jigsaw : 0.945312, class : 0.640625 [bs:128]
30/58 of epoch 20/30 jigsaw : 0.348661, class : 0.116904 - acc jigsaw : 0.882812, class : 0.601562 [bs:128]
Accuracies on val: jigsaw : 0.997590, class : 0.884337
Accuracies on test: jigsaw : 0.994012, class : 0.860479
New epoch - lr: 0.001
0/58 of epoch 21/30 jigsaw : 0.484477, class : 0.156196 - acc jigsaw : 0.851562, class : 0.546875 [bs:128]
30/58 of epoch 21/30 jigsaw : 0.232684, class : 0.045663 - acc jigsaw : 0.914062, class : 0.609375 [bs:128]
Accuracies on val: jigsaw : 0.997590, class : 0.885542
Accuracies on test: jigsaw : 0.995808, class : 0.864072
New epoch - lr: 0.001
0/58 of epoch 22/30 jigsaw : 0.197663, class : 0.154810 - acc j

Accuracies on test: jigsaw : 0.990419, class : 0.879042
New epoch - lr: 0.001
0/58 of epoch 12/30 jigsaw : 0.489259, class : 0.044207 - acc jigsaw : 0.867188, class : 0.679688 [bs:128]
30/58 of epoch 12/30 jigsaw : 0.331441, class : 0.277970 - acc jigsaw : 0.914062, class : 0.648438 [bs:128]
Accuracies on val: jigsaw : 0.996386, class : 0.891566
Accuracies on test: jigsaw : 0.995808, class : 0.871257
New epoch - lr: 0.001
0/58 of epoch 13/30 jigsaw : 0.423877, class : 0.287692 - acc jigsaw : 0.898438, class : 0.570312 [bs:128]
30/58 of epoch 13/30 jigsaw : 0.471905, class : 0.116678 - acc jigsaw : 0.867188, class : 0.554688 [bs:128]
Accuracies on val: jigsaw : 0.985542, class : 0.904819
Accuracies on test: jigsaw : 0.982036, class : 0.872455
New epoch - lr: 0.001
0/58 of epoch 14/30 jigsaw : 0.324682, class : 0.099623 - acc jigsaw : 0.929688, class : 0.609375 [bs:128]
30/58 of epoch 14/30 jigsaw : 0.376320, class : 0.092546 - acc jigsaw : 0.898438, class : 0.695312 [bs:128]
Accuracies 

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jig_weight in [0.0]:
    print("\n======================\n%g\n===================" % args.jig_weight)
    for args.bias_whole_image in [1.0]: # [None, 0.01, 0.05, 0.1, 0.3, 0.5]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % ("-".join(args.source),
                                                            args.target,args.epochs, args.jig_weight))

In [ ]:
dataloaders = {"train": data_helper.get_dataloader(args.source, args.jigsaw_n_classes, "train"),
           "val": data_helper.get_dataloader(args.target, args.jigsaw_n_classes, "val")}
dataset_sizes = {"train": len(dataloaders["train"].dataset),
                "val": len(dataloaders["val"].dataset)}
print(dataset_sizes)

In [ ]:
def get_optim_and_scheduler(network, epochs, lr):
    from torch import optim
    optimizer = optim.SGD(network.get_params(lr), weight_decay=.0005, momentum=.9, nesterov=True, lr=lr)
    step_size = int(epochs * .8)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)
    print(step_size)
    return optimizer, scheduler

model_ft = model_factory.get_network("caffenet")(jigsaw_classes=args.jigsaw_n_classes+1, classes=args.n_classes)
model_ft = model_ft.to(device)
# print(model_ft)

optimizer, scheduler = get_optim_and_scheduler(model_ft, args.epochs, args.learning_rate)

In [ ]:
def do_epoch(model, source, target, optimizer, logger, device):
    criterion = nn.CrossEntropyLoss()
    model.train()
    for it, ((data, jig_l, class_l), d_idx) in enumerate(source):
        data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
        
        optimizer.zero_grad()
        
        jigsaw_logit, class_logit = model(data)
        jigsaw_loss = criterion(jigsaw_logit, jig_l)
        class_loss = criterion(class_logit[d_idx!=target_id], class_l[d_idx!=target_id])
        _, cls_pred = class_logit.max(dim=1)
        _, jig_pred = jigsaw_logit.max(dim=1)
        loss = class_loss + jigsaw_loss * jig_weight
        
        loss.backward()
        optimizer.step()

        logger.log(it, len(source), {"jigsaw": jigsaw_loss.item(), "class": class_loss.item()},
                  {"jigsaw": torch.sum(jig_pred == jig_l.data).item(), "class":torch.sum(cls_pred == class_l.data).item()},
                  data.shape[0])
        del loss, class_loss, jigsaw_loss, jigsaw_logit, class_logit

    model.eval()
    with torch.no_grad():
        jigsaw_correct = 0
        class_correct = 0
        total = 0
        for it, ((data, jig_l, class_l), d_idx) in enumerate(target):
            data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
            jigsaw_logit, class_logit = model(data)
            _, cls_pred = class_logit.max(dim=1)
            _, jig_pred = jigsaw_logit.max(dim=1)
            class_correct += torch.sum(cls_pred == class_l.data)
            jigsaw_correct += torch.sum(jig_pred == jig_l.data)
            total += data.shape[0]
        logger.log_test({"jigsaw": float(jigsaw_correct) / total,
                         "class": float(class_correct) / total})


def do_training(args, model, source, target, optimizer, scheduler, device):
    logger = Logger(args)
    for k in range(args.epochs):
        scheduler.step()
        logger.new_epoch(scheduler.get_lr())
        do_epoch(model, source, target, optimizer, logger, device)
    return logger, model

In [ ]:
jig_weight = args.jig_weight
logger, model = do_training(args, model_ft, dataloaders["train"], dataloaders["val"], optimizer, scheduler, device)

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(args.source),args.target,args.epochs, jig_weight))

In [ ]:
def to_plt(inp):
    import numpy as np
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = model_ft.features[0] # models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(source),target,epochs, jig_weight))

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.show()

In [ ]:
for e,k in enumerate(range(0,l,int(updates))):
    print(k, logger.val_acc[e])

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
from data.data_helper import get_val_dataloader, get_train_dataloader
from os.path import join, dirname
# from data.JigsawLoader import JigsawTestDataset
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [ ]:
# loader = get_val_dataloader("photo",31,batch_size=10,multi=True)
loader, _ = get_train_dataloader(["photo"],31)

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    return inp

# dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)
# test = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
iter_c = iter(loader)
(tmp, v, c), d = next(iter_c)
print(tmp[:,0].shape)
for x in range(10):
#     image = tmp[0, x]
    image = torchvision.utils.make_grid(tmp[x],1,normalize=True)
    plt.imshow(to_plt(image))
    plt.show()
    print(v[x])

# print(tmp[:,0].shape)
# for x in range(tmp.shape[1]):
# #     image = tmp[0, x]
#     image = torchvision.utils.make_grid(tmp[0, x],1,normalize=True)
#     plt.imshow(to_plt(image))
#     plt.show()
#     print(v[0,x])

    
# print(v.max(), v.min())

In [ ]:
res = []
for k in range(10):
    res.append(torch.rand(5, 100))
res = torch.stack(res,0)
res.mean(1).shape

In [ ]:
from data.JigsawLoader import JigsawDataset
from PIL import Image
import torchvision.transforms as transforms

class JigsawTestDataset(JigsawDataset):
    def __init__(self, *args, **xargs):
        super().__init__(*args, **xargs)
        self._augment_tile = transforms.Compose([
#             transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]
        img = Image.open(framename).convert('RGB')
        img = self._image_transformer(img)

        w = float(img.size[0]) / self.grid_size
        n_grids = self.grid_size ** 2
        tiles = [None] * n_grids
        for n in range(n_grids):
            y = int(n / self.grid_size)
            x = n % self.grid_size
            tile = img.crop([x * w, y * w, (x + 1) * w, (y + 1) * w])
            tile = self._augment_tile(tile)
            tiles[n] = tile

        data = torch.stack(tiles, 0)
        return self.returnFunc(data), 0, int(self.labels[index])


In [ ]:
dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)

In [ ]:
import torch
import random

In [ ]:
true = 0
for x in range(100):
    true += 0.1 > random.random()
true